# Ray RLlib Multi-Armed Bandits - Picking an Exploration-Exploitation Strategy

© 2019-2020, Anyscale. All Rights Reserved

![Anyscale Academy](../../images/AnyscaleAcademy_Logo_clearbanner_141x100.png)

What strategy should we follow for selecting actions that balance the exploration-exploitation tradeoff, yielding the maximum average reward over time?

This is the essence of most RL/MAB algorithms. Let's briefly discuss four algorithms, then explore two of them with examples over several subsequent lessons.

> **Tip:** Understanding the rest of this lesson is not required for the subsequent lessons. So, you can safely skip to the [next lesson](03-Simple-Multi-Armed-Bandit.ipynb) if you prefer. Consider returning here later when you need a better understanding of how these algorithms work. 

However, you might skim the $\epsilon$-Greedy section and the first part of the Upper Confidence Bound section (up to the _In more detail_ part), to get a sense of how decisions can be made.

### $\epsilon$-Greedy

One possible strategy is $\epsilon$-Greedy, where $\epsilon$ is small number that determines how frequently exploration is done. The best-known action is exploited most of the time, governed by probability $1 - \epsilon$ (i.e., in percentage terms $100*(1 - \epsilon)$%). With probability $\epsilon$, a previously-unexplored action is randomly picked in the hopes of finding a new, best-rewarding action.

Typical values of $\epsilon$ are between 0.01 and 0.1. A larger value, like 0.1, explores more aggressively and finds the optimal policy more quickly, but afterwards the aggressive exploration strategy becomes a liability, as it only selects the optimal action ~90% of the time, continuing excessive exploration that is now counterproductive. In contrast, smaller values, like 0.01, are slower to find the optimal policy, but once found continue to select it ~99% of the time, so over time the mean reward is _higher_ for _smaller_ $\epsilon$ values, as the optimal action is selected more often.

See [Wikipedia - MAB Approximate Solutions](https://en.wikipedia.org/wiki/Multi-armed_bandit) and [Sutton 2018](https://mitpress.mit.edu/books/reinforcement-learning-second-edition) for more information. 

### Upper Confidence Bound

A limitation about $\epsilon$-greedy is that exploration is done indiscriminately. Is it possible to make a more informed choice about which alternative actions are more likely to yield a good result, so we preferentially pick one of them? That's what the Upper Confidence Bound (UCB) algorithm attempts to do. It weights some choices over others. 

It's worth looking at the formula that governs the choice for the next action at time $t$:

$$A_t \doteq \frac{argmax}{a}\bigg[ Q_t(a) + c\sqrt{ \dfrac{\ln(t)}{N_t(a)} }\bigg]$$

It's not essential to fully understand all the details, but here is the gist of it; the best action to take at time $t$, $A_t$, is decided by picking the best known action for returning the highest value (the $Q_t(a)$ term in the brackets [...] computes this), but with a correction term that encourages exploration, especially for smaller $t$, but less so for a particular $a$ if we've already picked it a lot (the second term).

#### In more detail

* $A_t$ is the action we want to select at time $t$, the action that is most likely to produce the best reward or most likely to be worth exploring.
* For all the actions we can choose from, we pick the action $a$ that maximizes the formula in the brackets [...].
* $Q_t(a)$ is any equation we're using to measure the "value" received at time $t$ for action $a$. This is the greedy choice, i.e., the equation that tells us which action $a$ we currently know will give us the highest value. If we never wanted to explore, the second term in the brackets wouldn't exist. $Q_t(a)$ alone would always tell us to pick the best action we already know about. 
* The second term in the brackets is the correction that UCB gives us. As time $t$ increases, the natural log of $t$ also increases, but slower and slower for larger $t$. This is good because we hope we will find the real best action at some earlier time $t$, so exploration at large $t$ is less useful. However, the denominator, $N_t(a)$ is the number of times we've selected $a$ already. The more times we've already tried, the less "interesting" it is to try again, so this term penalizes choosing $a$. Finally, $c$ is a constant, a "knob" or _hyperparameter_ that determines how much we weight exploration vs. exploitation.


When we use UCB shortly, we'll use a simple _linear_ equation for $Q_t(a)$, i.e., something of the form $z = ax + by + c$.

See [Wikipedia - MAB Approximate solutions for contextual bandit](https://en.wikipedia.org/wiki/Multi-armed_bandit), [these references](../06-RL-References.ipynb#Upper-Confidence-Bound), and the [RLlib documentation](https://docs.ray.io/en/latest/rllib-algorithms.html?highlight=greedy#linear-upper-confidence-bound-contrib-linucb) for more information. 

### Thompson Sampling

Thompson sampling, developed in the 30s, is similar to UCB in that it picks the action that is believed to have the highest potential of maximum reward. The probability of selecting any of the available actions is based on the expectation of reward from the posterior distributions of rewards with previously sampled actions and contexts rewards.

For more information, see [Wikipedia](https://en.wikipedia.org/wiki/Thompson_sampling), [A Tutorial on Thompson Sampling](https://web.stanford.edu/~bvr/pubs/TS_Tutorial.pdf), [RLlib documentation](https://docs.ray.io/en/latest/rllib-algorithms.html?highlight=greedy#linear-thompson-sampling-contrib-lints), and other references in [06 RL References](../06-RL-References.ipynb).

### Gradient Bandit Algorithms

Focusing explicitly on rewards isn't the only approach. What if we use a more general measure, a _preference_, for selecting an action $a$ at time $t$, represented here $H_t(a)$? We need to model this so we have a probability of selecting an action $a$. Using the _soft-max distribution_ works, also known as the Gibbs or Boltzmann distribution:

$Pr\{A_t = a\} \doteq \frac{e^{H_t(a)}}{\sum^{k}_{b=1}e^{H_t(b)}} \doteq \pi_t(a)$

$\pi_t(a)$ is defined to encapsulate this formula for the probability of taking action $a$ at time $t$.

The term _gradient_ is used for this algorithm because the training update formula for $H_t(a)$ is very similar to the _stochastic gradient descent_ formula used in other ML problems. 

After an action $a$ is selected at a time $t$ and reward $R_t$ is received, the action preferences are updated as follows:

$ H_{t+1}(A_t) \doteq H_t(A_t) + \alpha(R_t - \overset{\_}{R_t})(1 - \pi_t(A_t))$,  and

$ H_{t+1}(a) \doteq H_t(a) - \alpha(R_t - \overset{\_}{R_t})(\pi_t(a))$,  for all $a \ne A_t$

where $\alpha > 0$ is a step size parameter and $\overset{\_}{R_t}$ is the average of all the rewards up through and including time $t$. Note that if $R_t - \overset{\_}{R_t}$ is positive, meaning the current reward is larger than the average, the preference $H(A_t)$ increases. Otherwise, it descreases. Note the plus vs. minus signs in the two equations before the $\alpha$ term. We would expect the _other_ $H(a)$ terms, for the rest of the $a$ actions, to decrease when $H(A_t)$ increases. That is, if our preference for $A_t$ increases, our preferences for the other actions should decrease.

We won't discuss this algorithm further. See [Sutton 2018](https://mitpress.mit.edu/books/reinforcement-learning-second-edition) for more details.